In [78]:
def generate_key_matrix():
    # checked to be invertible
    return [[2,3], [5,7]]

key_matrix = generate_key_matrix()
print(key_matrix)


[[2, 3], [5, 7]]


In [79]:
def to_number(text):
    return [ord(char) - 64 for char in text]

def to_text(numbers):
    return [chr(num + 64) for num in numbers]

print(to_number('ABCDEFGHIJKLMNOPQRSTUVWXYZ'))

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26]


In [80]:
def chunk_pad(numbers, key_matrix):
    # key matrix is a cube, so we need to chunk the plain text in chunks of len(key_matrix)
    chunk_size = len(key_matrix)
    # code for padding in ASCII
    # in this case we use capital A for simplicity
    space = ord('A') - 64
    # add spaces until it becomes divisible by chunk_size
    while len(numbers) % chunk_size != 0:
        numbers.append(space)
    #print(numbers)

    arrays = [numbers[i:i+chunk_size] for i in range(0, len(numbers), chunk_size)]
    #print(arrays)
    return arrays

def collect(numbers):
    # opposite operation
    return ''.join([''.join([chr(num + 64) for num in chunk]) for chunk in numbers])

print(chunk_pad(to_number('ABCDEFGHIJKLMNOPQRSTUVWXYZ'), key_matrix))
# spaces are not visible
print(collect(chunk_pad(to_number('ABCDEFGHIJKLMNOPQRSTUVWXYZ'), key_matrix)))

[[1, 2], [3, 4], [5, 6], [7, 8], [9, 10], [11, 12], [13, 14], [15, 16], [17, 18], [19, 20], [21, 22], [23, 24], [25, 26]]
ABCDEFGHIJKLMNOPQRSTUVWXYZ


In [83]:
import numpy as np # for matrix inversion

def encrypt(plain_text,key_matrix):
    # to encrypt the plain text, we need to convert it to a list of numbers first.
    plain_text_list = to_number(plain_text)
    cipher_text_list = []
    
    # we are performing Hill Cipher encryption here
    # element = plain_text_list[i]
    # now im kinda stuck until i figure out how to multiply the matrix in python with no library beacuse I didn't see if it was allowed or not
    # we need to spot the pattern
    # key_matrix[0][0] * plain_text_list[0] + key_matrix[0][1] * plain_text_list[1] + key_matrix[0][2] * plain_text_list[2] + key_matrix[0][3] * plain_text_list[3]
    # key_matrix[1][0] * plain_text_list[0] + key_matrix[1][1] * plain_text_list[1] + key_matrix[1][2] * plain_text_list[2] + key_matrix[1][3] * plain_text_list[3]
    # key_matrix[2][0] * plain_text_list[0] + key_matrix[2][1] * plain_text_list[1] + key_matrix[2][2] * plain_text_list[2] + key_matrix[2][3] * plain_text_list[3]
    # we are currently performing one row in the matrix multiplication every time
    # now we need to spot the pattern...
    # I notice that the inner digit of key_matrix is moving more quickly than the outer one, that means we need a double for loop
    # now many times are we looping? 
    # first thing we have to do is split the input into chunks and then pad it
    plain_chunks = chunk_pad(plain_text_list, key_matrix)
    # now it's easy game, i just need to figure out how to do it for one padded chunk
    cipher_text_list = [] # the output is gonna be of the same length as the input
    for i in range(len(plain_chunks)):
        cipher_text_list.append([])
        # now we have the matrix multiplication of the plain text chunk and the key matrix
        chunk_size = len(key_matrix)
        for j in range(chunk_size):
            cipher_text_list[i].append(0)
            for k in range(chunk_size):
                # k is looping fast, so that's the inner part of the matrix.
                # the ith chunk, so we get the ith element from plain_chunks
                cipher_text_list[i][j] += key_matrix[j][k] * plain_chunks[i][k]
            # they will get out of range, so we need to modulo them by plaintext allowed chars length, in our case we have 26 only, covering just the letters
            cipher_text_list[i][j] %= 26
    print(cipher_text_list)
    return collect(cipher_text_list)



 # Your code goes here
def decrypt(cipher_text,key_matrix):
    # The decryption process is similar, only uses an inverse key matrix.
    # we don't have a library, so we have to do it ourselves.

    inv_matrix = np.linalg.inv(key_matrix)
    # Convert cipher text to a list of numbers
    cipher_text_list = to_number(cipher_text)
    # Split the cipher text into chunks
    cipher_chunks = chunk_pad(cipher_text_list, key_matrix)
    plain_text_list = []  # This will hold the decrypted numeric values
    
    for i in range(len(cipher_chunks)):
        plain_text_list.append([])
        chunk_size = len(inv_matrix)
        for j in range(chunk_size):
            plain_text_list[i].append(0)
            for k in range(chunk_size):
                # Perform matrix multiplication using the inverse key matrix
                
                plain_text_list[i][j] += int(inv_matrix[j][k]) * cipher_chunks[i][k]
            # Apply modulo to keep within the range of allowed characters
            plain_text_list[i][j] %= 26
    
    # Convert the numeric list back to text
    print(plain_text_list)
    decrypted_text = collect(plain_text_list)
    return decrypted_text

# Your code goes here
plain_text="TEST" # 7 chars

print("------------------------")
print("original plain text: ",plain_text)
cipher_text=encrypt(plain_text,key_matrix)
print("cipher text: ", cipher_text)
print("plain text: ", decrypt(cipher_text, key_matrix))

# Example input and output:

#plaintext entered= "TEST"
#ciphertext= "OSEJ"


------------------------
original plain text:  TEST
[[3, 5], [20, 1]]
cipher text:  CETA
[[20, 5], [19, 20]]
plain text:  TEST
